In [1]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

--2019-12-29 19:56:47--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: ‘moviedataset.zip’

100%[======================================>] 160,301,210 35.1MB/s   in 4.4s   

2019-12-29 19:56:52 (34.6 MB/s) - ‘moviedataset.zip’ saved [160301210/160301210]

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: tags.csv                


# Preprocessing

In [2]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [84]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [85]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [86]:
ratings_df.sort_values(by=['userId'], inplace=True)

In [19]:
ratings_df.loc[ratings_df['userId']==3]

,userId,movieId,rating
9,3,2431,5.0
10,3,2445,4.0
8,3,2394,4.0
7,3,356,4.0


In [20]:
movies_df['year'] = movies_df.title.str.extract('(\d\d\d\d)',expand=False)
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [21]:
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [22]:
moviesWithGenres_df = movies_df.copy()

for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1

moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Part 1: Content-Based recommendation system

In [58]:
inputMovies=ratings_df.loc[ratings_df['userId']==3]
inputMovies.reset_index(drop=True)
inputMovies

,userId,movieId,rating
9,3,2431,5.0
10,3,2445,4.0
8,3,2394,4.0
7,3,356,4.0


In [59]:
userMovies = pd.concat([inputMovies, moviesWithGenres_df], axis=1, join='inner')
userMovies

,userId,movieId,rating,movieId,title,genres,year,Adventure,Animation,Children,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
9,3,2431,5.0,10,GoldenEye,"[Action, Adventure, Thriller]",1995,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,3,2445,4.0,11,"American President, The","[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,3,2394,4.0,9,Sudden Death,[Action],1995,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,3,356,4.0,8,Tom and Huck,"[Adventure, Children]",1995,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
inputMovies['rating']

9     5.0
10    4.0
8     4.0
7     4.0
Name: rating, dtype: float64

In [61]:
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1).drop('userId', 1).drop('rating', 1)

In [62]:
userGenreTable.astype(float)

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
userGenreTable.dtypes

Adventure             float64
Animation             float64
Children              float64
Comedy                float64
Fantasy               float64
Romance               float64
Drama                 float64
Action                float64
Crime                 float64
Thriller              float64
Horror                float64
Mystery               float64
Sci-Fi                float64
IMAX                  float64
Documentary           float64
War                   float64
Musical               float64
Western               float64
Film-Noir             float64
(no genres listed)    float64
dtype: object

In [64]:
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
userProfile

Adventure             9.0
Animation             0.0
Children              4.0
Comedy                4.0
Fantasy               0.0
Romance               4.0
Drama                 4.0
Action                9.0
Crime                 0.0
Thriller              5.0
Horror                0.0
Mystery               0.0
Sci-Fi                0.0
IMAX                  0.0
Documentary           0.0
War                   0.0
Musical               0.0
Western               0.0
Film-Noir             0.0
(no genres listed)    0.0
dtype: float64

In [65]:
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
#And drop the unnecessary information
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())


In [74]:

recommendationTable_df = recommendationTable_df.sort_values(ascending=False)

recommendationTable_df.head()

movieId
83266    0.897436
4956     0.897436
31367    0.794872
74404    0.794872
74406    0.794872
dtype: float64

In [75]:
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

,movieId,title,genres,year
376,380,True Lies,"[Action, Adventure, Comedy, Romance, Thriller]",1994
455,459,"Getaway, The","[Action, Adventure, Crime, Drama, Romance, Thr...",1994
4705,4800,King Solomon's Mines,"[Action, Adventure, Drama, Romance, Thriller]",1937
4861,4956,"Stunt Man, The","[Action, Adventure, Comedy, Drama, Romance, Th...",1980
6455,6564,Lara Croft Tomb Raider: The Cradle of Life,"[Action, Adventure, Comedy, Romance, Thriller]",2003
7839,8481,Northwest Passage,"[Action, Adventure, Drama, Romance, Thriller, ...",1940
9296,27344,Revolutionary Girl Utena: Adolescence of Utena...,"[Action, Adventure, Animation, Comedy, Drama, ...",1999
9459,27735,Unstoppable,"[Action, Adventure, Comedy, Drama, Thriller]",2004
9697,31367,"Chase, The","[Action, Adventure, Comedy, Crime, Romance, Th...",1994
10298,34435,Sholay,"[Action, Adventure, Comedy, Musical, Romance, ...",1975


# Part 2: Collaborative Filtering

In [88]:
inputMovies=ratings_df.loc[ratings_df['userId']==3]
inputMovies.reset_index(drop=True)
inputMovies

,userId,movieId,rating
10,3,2445,4.0
9,3,2431,5.0
7,3,356,4.0
8,3,2394,4.0


In [92]:
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset = userSubset.loc[userSubset['userId']!=3]
userSubset.head(20)

,userId,movieId,rating
21,4,356,2.0
355,11,2431,2.5
577,13,2394,3.0
692,14,356,5.0
786,15,356,4.5
2049,17,2431,3.0
2023,17,2394,3.0
1352,17,356,5.0
2292,18,356,3.0
2796,23,2394,4.0


In [94]:
userSubsetGroup = userSubset.groupby(['userId'])
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)
userSubsetGroup [:2]

[(774,        userId  movieId  rating
  69779     774     2445     1.0
  69484     774      356     5.0
  69772     774     2394     4.0
  69777     774     2431     3.0), (1250,         userId  movieId  rating
  115209    1250      356     4.0
  115457    1250     2445     3.0
  115455    1250     2431     5.0
  115446    1250     2394     3.0)]

In [95]:
userSubsetGroup=userSubsetGroup[:100]

In [96]:
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [107]:
pearsonCorrelationDict

{774: -0.09759000729485333,
 1250: 0.8703882797784892,
 1643: -0.9304340030613443,
 1658: -0.7745966692414834,
 4208: -0.5555555555555556,
 4415: 0.0,
 7734: -0.9428090415820635,
 9797: -0.7745966692414834,
 14572: -0.3779644730092272,
 16484: -0.47140452079103173,
 18506: 0.0,
 19817: -0.17407765595569785,
 22883: -0.37582301400141443,
 23216: 0.6622661785325219,
 26125: -0.13245323570650439,
 26706: -0.3333333333333333,
 31302: 0.5222329678670935,
 32417: 0.5222329678670935,
 33121: -0.13245323570650439,
 36946: 0.050443327230531826,
 37356: 0.0,
 38318: -0.2581988897471611,
 38754: 0.19245008972987526,
 42196: -0.5773502691896258,
 45300: 0,
 50060: -0.7777777777777778,
 50475: 0.37582301400141443,
 50544: -0.17407765595569785,
 51287: 0.3651483716701107,
 51580: 0.3333333333333333,
 52974: 0.5555555555555556,
 53498: -0.3333333333333333,
 56395: 0.5222329678670935,
 56493: 0.0,
 58040: -0.5222329678670935,
 61095: 0.5222329678670935,
 61154: -0.3333333333333333,
 61750: -0.52223296

In [108]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,-0.097590,774
1,0.870388,1250
2,-0.930434,1643
3,-0.774597,1658
4,-0.555556,4208


In [113]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:30]
topUsers.tail()

,similarityIndex,userId
93,0.258199,194974
22,0.192450,38754
97,0.174078,200453
51,0.132453,96701
19,0.050443,36946


In [114]:
topUsersRating= topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head() 

,similarityIndex,userId,movieId,rating
0,0.870388,97610,2375,4.0
1,0.870388,97610,2369,4.0
2,0.870388,97610,2355,4.0
3,0.870388,97610,2335,3.0
4,0.870388,97610,2346,3.0


In [115]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,0.870388,97610,2375,4.0,3.481553
1,0.870388,97610,2369,4.0,3.481553
2,0.870388,97610,2355,4.0,3.481553
3,0.870388,97610,2335,3.0,2.611165
4,0.870388,97610,2346,3.0,2.611165


In [116]:
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,13.813043,56.152266
2,11.930004,38.472608
3,6.278974,20.297376
4,5.468628,18.213320
5,9.599340,32.974980


In [117]:
recommendation_df = pd.DataFrame()
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,4.065163,1
2,3.224861,2
3,3.232594,3
4,3.330510,4
5,3.435130,5


In [118]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
5615,5.0,5615
116,5.0,116
54686,5.0,54686
91886,5.0,91886
92234,5.0,92234
92335,5.0,92335
74624,5.0,74624
92483,5.0,92483
456,5.0,456


In [124]:
recommendation_df.index = range(len(recommendation_df))

In [129]:
recommendation_df.merge(movies_df,left_on='movieId',right_on='movieId' ,how='inner') 

,weighted average recommendation score,movieId,title,genres
0,5.0,5615,Invincible (2001),Drama
1,5.0,116,Anne Frank Remembered (1995),Documentary
2,5.0,54686,"Last Legion, The (2007)",Action|Adventure|Fantasy|War
3,5.0,91886,Dolphin Tale (2011),Children|Drama
4,5.0,92234,Red Tails (2012),Action|Adventure|Drama|War
5,5.0,92335,Big Miracle (2012),Drama|Romance
6,5.0,74624,Agora (2009),Adventure|Drama|Romance
7,5.0,92483,Living Proof (2008),Drama
8,5.0,456,Fresh (1994),Crime|Drama|Thriller
9,5.0,93132,Seven Days in Utopia (2011),Drama
